### Option 1 - Multiprocessing

In [1]:
import multiprocessing

def main(customRobot):
  turn_count = 0
  length_of_arena = 120 #cm - change laer
  # Based on robot starting in bottom left corner

  while True: ### MADE CHANGE HERE ###
    # DRIVE ROBOT
    if turn_count ==0:
      dist = length_of_arena
      customRobot.drive_robot(dist, "cm", "forward") # dist variable can be removed, it is just a safety net if the ultrasonics don't go off
      # constantly run customRobot.ultrasonic_sensor() in parallel here

    elif turn_count ==1:
      dist = customRobot.distance_to_bin() #check distance from wall
      customRobot.drive_robot(dist, "cm", "forward")
      # constantly run customRobot.ultrasonic_sensor() in parallel here
      customRobot.release_lever()
      customRobot.drive_robot(length_of_arena-dist, "cm", "forward")

    elif turn_count == 2:
      dist = length_of_arena
      customRobot.drive_robot(dist, "cm", "forward")
      # constantly run customRobot.ultrasonic_sensor() in parallel here

    elif turn_count == 3:
      dist = length_of_arena
      customRobot.drive_robot(dist, "cm", "forward")
      # constantly run customRobot.ultrasonic_sensor() in parallel here

if __name__ == "__main__":
  customRobot = CustomRobot() ### MADE CHANGE HERE ###

  main_process = multiprocessing.Process(target=main, args=(customRobot,)) # so continuously drive out bot
  sensor_process = multiprocessing.Process(target=customRobot.ultrasonic_sensor) # continuously check sensors

  main_process.start()
  sensor_process.start()

  main_process.join()
  sensor_process.join()


In [ ]:
### Robot
import numpy as np
import RPi.GPIO as GPIO
import sys, os, time, math
import encoder
from encoder import Encoder

from gpiozero import RotaryEncoder, Robot, OutputDevice, PWMOutputDevice

class CustomRobot:
    def __init__(self, left_motor_pins, right_motor_pins, trig_1, echo_1, echo_2, echo_3, echo_4, left_encoder_pins=None, right_encoder_pins=None, distance_per_step=None,):
        self.left_motor_pins = left_motor_pins # PWM out
        self.right_motor_pins = right_motor_pins # PWM out
        self.left_encoder = RotaryEncoder(self.left_encoder_pins, max_steps=1000000)
        self.right_encoder = RotaryEncoder(self.right_encoder_pins, max_steps=1000000)

        # Ultrasonic Sensor
        self.TRIG_1 = trig_1
        self.ECHO_1 = echo_1
        self.ECHO_2 = echo_2
        self.ECHO_3 = echo_3
        self.ECHO_4 = echo_4

        self.distance_per_step = distance_per_step

        self.right_count = 0
        self.left_count = 0

        self.stop_event = multiprocessing.Event()

    def drive_robot(self, distance, units, direction):
        '''
        Function to set Robot's velocity.

        Inputs:
        - velocity: List, first element is forward velocity and second element is rotational velocity
        - time: Time in seconds to apply velocity
        '''

        # Distance to Steps Conversion
        one_step_in_cm = 3 #cm #TO Calibrate
        steps_needed = distance/one_step_in_cm

        # Need to get steps.right_encoder fixed - TO check, both encoders are giving same amount of steps in the same distance
        self.right_encoder.steps = 0
        while self.right_encoder.steps < steps_needed and not self.stop_event.is_set(): ### MADE CHANGE HERE ### -> NOTE THIS CODE SHOULD WORK IF WE ARE USING ENCODER FEEDBACK TO CHECK FOR THIS CONDITION, RATHER THAN USING TIME.SLEEP() TO DRIVE FOR A GIVEN TIME
          if direction == "forward":
              GPIO.output(self.left_motor_pins[0], GPIO.HIGH)
              GPIO.output(self.left_motor_pins[1], GPIO.LOW)
              GPIO.output(self.right_motor_pins[0], GPIO.LOW)
              GPIO.output(self.right_motor_pins[1], GPIO.HIGH)
          elif direction == "backwards": # Backwards
              GPIO.output(self.left_motor_pins[0], GPIO.LOW)
              GPIO.output(self.left_motor_pins[1], GPIO.HIGH)
              GPIO.output(self.right_motor_pins[0], GPIO.HIGH)
              GPIO.output(self.right_motor_pins[1], GPIO.LOW)
          elif direction == "right": # Right
              GPIO.output(self.left_motor_pins[0], GPIO.HIGH)
              GPIO.output(self.left_motor_pins[1], GPIO.LOW)
              GPIO.output(self.right_motor_pins[0], GPIO.HIGH)
              GPIO.output(self.right_motor_pins[1], GPIO.LOW)
          elif direction == "left": # Left
              GPIO.output(self.left_motor_pins[0], GPIO.LOW)
              GPIO.output(self.left_motor_pins[1], GPIO.HIGH)
              GPIO.output(self.right_motor_pins[0], GPIO.LOW)
              GPIO.output(self.right_motor_pins[1], GPIO.HIGH)

        #time.sleep(duration)
        self.stop_drive()

    def collect_ultrasonic_sensor_distances(self):
       while True:
        #Sensor 1
          GPIO.output(self.TRIG_1, False)
          time.sleep(0.1) #0.2
          GPIO.output(self.TRIG_1, True)
          time.sleep(0.2) #1
          GPIO.output(self.TRIG_1, False)
          while GPIO.input(self.ECHO_1)==0:
            pulse_start=time.time()
          while GPIO.input(self.ECHO_1)==1:
            pulse_end=time.time()
          pulse_duration=pulse_end-pulse_start
          sensor_1_distance=pulse_duration*17150
          if sensor_1_distance<5:
            break

        #Sensor 2
          GPIO.output(self.TRIG_1, False)
          #time.sleep(0.1)
          GPIO.output(self.TRIG_1, True)
          #time.sleep(0.1)
          GPIO.output(self.TRIG_1, False)
          while GPIO.input(self.ECHO_2)==0:
            pulse_start=time.time()
          while GPIO.input(self.ECHO_2)==1:
            pulse_end=time.time()
          pulse_duration=pulse_end-pulse_start
          sensor_2_distance=pulse_duration*17150
          if sensor_2_distance<5:
            break

        #Sensor 3
          GPIO.output(self.TRIG_1, False)
          GPIO.output(self.TRIG_1, True)
          GPIO.output(self.TRIG_1, False)
          while GPIO.input(self.ECHO_3)==0:
            pulse_start=time.time()
          while GPIO.input(self.ECHO_3)==1:
            pulse_end=time.time()
          pulse_duration=pulse_end-pulse_start
          sensor_3_distance=pulse_duration*17150
          if sensor_3_distance<5:
            break

        #Sensor 4
          GPIO.output(self.TRIG_1, False)
          #time.sleep(0.1)
          GPIO.output(self.TRIG_1, True)
          #time.sleep(0.1)
          GPIO.output(self.TRIG_1, False)
          while GPIO.input(self.ECHO_4)==0:
            pulse_start=time.time()
          while GPIO.input(self.ECHO_4)==1:
            pulse_end=time.time()
          pulse_duration=pulse_end-pulse_start
          sensor_4_distance=pulse_duration*17150
          if sensor_4_distance<5:
            break

       return sensor_1_distance, sensor_2_distance,sensor_3_distance, sensor_4_distance

    def ultrasonic_sensor(self, s1, turn_count):
      while not self.stop_event.is_set(): ### MADE CHANGE HERE ###
        sensor_1_distance, sensor_2_distance, sensor_3_distance, sensor_4_distance = self.collect_ultrasonic_sensor_distances()

        if (sensor_2_distance <5 and sensor_3_distance <5 and sensor_4_distance <5):
          self.stop_event.set() ### MADE CHANGE HERE ###

          # above line should stop drive
          # self.stop_drive()

          self.stop_event.clear() ### MADE CHANGE HERE ###

          # add condition to distinguish wall areas - based on step count
          self.drive_robot(90, "deg", "right") # calibrate to take into consideration the not being parallel to wall
          turn_count += 1

        elif (sensor_1_distance < 5):
          self.stop_event.set() ### MADE CHANGE HERE ###

          # self.stop_drive()

          self.drive_robot(5, "deg", "right")

        elif (sensor_1_distance < 6): # check tolerance we want
          self.stop_event.set() ### MADE CHANGE HERE ###

          # self.stop_drive()

          self.drive_robot(5, "deg", "left")

      def distance_to_bin():
        # write code to converse QR to number 1 2 3
        bin_number = 1

        if bin_number ==1:
          distance_to_bin = 30 # double check - need to minus 5 potentially based on how far robot it to edge or can be calculated outside function
        elif bin_number==2:
          distance_to_bin = 60
        elif bin_number==3:
          distance_to_bin = 90
        return distance_to_bin


      def release_lever():
          pass

      def stop_drive(self):
          GPIO.output(self.left_motor_pins[1], GPIO.LOW)
          GPIO.output(self.left_motor_pins[0], GPIO.LOW)
          GPIO.output(self.right_motor_pins[1], GPIO.LOW)
          GPIO.output(self.right_motor_pins[0], GPIO.LOW)
          time.sleep(1)

### Option 2 - Threading

Basically same code, however instead of different processes (e.g sensor on CPU1, main on CPU2), we spawn threads on the same CPU process. Its lightweight, but not true parallelisation.